In [ ]:
#version 3
def average_panels(days, mice, binary_mask, shared_path, flash, success_fail):
    # Initialize a list to store the averaged data for each day
    average_across_mice = []

    # Define the folder path (excluding the day subfolder)
    folder_path_full = os.path.join(shared_path, flash, success_fail, "full_frame")

    for day in days:
        time_series_data = []  # List to store time series data for all mice for this particular day

        for mouse in mice:
            # Construct file path (without day in the folder structure)
            file_path_full = f"{folder_path_full}/{mouse}_stage2_{day}_{flash}_{success_fail}_ff.pickle"

            try:
                # Load data and compute z-score
                one_day_full = load_one_day_full(file_path_full)
                z_score_one_day_full = zscore(one_day_full, axis=1)

                # Append the time series data for this mouse
                time_series_data.append(z_score_one_day_full)

            except FileNotFoundError:
                print(f"File not found for mouse {mouse} on day {day}")
                if time_series_data:
                    # Append an array of zeros matching the existing shape if we have other data
                    time_series_data.append(np.zeros_like(time_series_data[0]))
                else:
                    print(f"No reference data available to generate zero array for missing mouse {mouse}.")
                    
            time_series_data = np.array(time_series_data)
            average_for_day = np.mean(time_series_data, axis=0)  # Average across mice

            # Store the averaged data for this day
            average_across_mice.append((average_for_day, day))

    return average_across_mice

In [ ]:
def plot_average_separate_panels(mice, filename, flash):
    # Define frames and their corresponding times
    frames_to_plot = [1, 8, 15, 22, 30, 38, 45, 52, 60]
    frames_to_seconds = [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]

    # Loop through the averaged data for each day
    for average_for_day, day in mice:
        print(f"Plotting for {day}")

        if len(average_for_day.shape) == 3:  # Check if it's a 3D array
            cropped_z_score_single_full = average_for_day[:, :-14, 7:-7]
            masked_z_score_single_full = cropped_z_score_single_full * binary_mask

            # Create a figure and subplots
            fig, axes = plt.subplots(1, 9, figsize=(20, 10))

            # Add a title for the entire figure
            fig.suptitle(f"{day} - {flash} flash ({filename})", 
                         color='white', fontsize=30, weight='bold', y=0.7)

            # Plot the specific frames
            for i, ax in enumerate(axes.flatten()):
                if i < len(frames_to_plot):
                    frame_index = frames_to_plot[i]
                
                    # Check if the frame_index is within bounds
                    if frame_index < masked_z_score_single_full.shape[0]:
                        image = masked_z_score_single_full[frame_index]
                        ax.imshow(image, cmap='RdYlBu_r')
                        fig.set_facecolor('#5D6D7D')  # Set background color to black
                        ax.axis('off')
                        ax.set_title(f'{frames_to_seconds[i]} s', color='white', fontsize=40)
                    else:
                        ax.axis('off')  # Hide the axes if the frame index is out of bounds
                else:
                    ax.axis('off')  # Hide the axes for unused subplots

            # Adjust layout
            plt.tight_layout()

            # Save the figure
            #save_file_path = f"X:/RaymondLab/1_Kai/3_processed_data/8_panels/{filename}_{flash}/{filename}_{day}_avg_brain_panels.png"
            #plt.savefig(save_file_path, facecolor=fig.get_facecolor(), bbox_inches='tight')

            # Optionally, you can show the figure if you want
            plt.show()

            # Close the figure to free up memory
            plt.close(fig)
